In [ ]:
import cv2
import time
from ultralytics import YOLO

def main():
    # Load custom trained model
    model = YOLO("runs/detect/train2/weights/best.pt")  # Your trained model
    
    # Video capture setup
    cap = cv2.VideoCapture(0)  # Use 0 for default camera, or video path
    if not cap.isOpened():
        print("Error: Could not open video source")
        return

    # Performance tracking
    frame_count = 0
    start_time = time.time()
    fps = 0
    
    # Detection parameters
    conf_threshold = 0.5  # Confidence threshold
    classes_to_detect = None  # None for all classes (modify if needed)

    while True:
        # Read frame
        ret, frame = cap.read()
        if not ret:
            break

        # Perform detection
        results = model.track(
            source=frame,
            imgsz=640,
            conf=conf_threshold,
            device=0,  # 0 for GPU, 'cpu' for CPU
            classes=classes_to_detect,
            persist=True,  # For tracking between frames
            verbose=False
        )

        # Get annotated frame
        annotated_frame = results[0].plot()

        # Calculate FPS
        frame_count += 1
        if frame_count >= 30:  # Update FPS every 30 frames
            fps = frame_count / (time.time() - start_time)
            frame_count = 0
            start_time = time.time()

        # Display FPS
        cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Show results
        cv2.imshow("Basketball Analytics - Real Time", annotated_frame)

        # Exit on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Cleanup
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

multicam track

In [ ]:
from threading import Thread

class VideoStream:
    def __init__(self, src=0):
        self.stream = cv2.VideoCapture(src)
        (self.grabbed, self.frame) = self.stream.read()
        self.stopped = False

    def start(self):
        Thread(target=self.update, args=()).start()
        return self

    def update(self):
        while not self.stopped:
            if not self.grabbed:
                self.stop()
            else:
                (self.grabbed, self.frame) = self.stream.read()

    def read(self):
        return self.frame

    def stop(self):
        self.stopped = True
        self.stream.release()